In [1]:
from transformers import AutoProcessor, Blip2ForConditionalGeneration
import torch

In [2]:
# pip install accelerate bitsandbytes
import torch
import requests
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration

# model_name="Salesforce/blip2-flan-t5-xl"
model_name="Salesforce/blip2-opt-6.7b-coco"
processor = Blip2Processor.from_pretrained(model_name)
model = Blip2ForConditionalGeneration.from_pretrained(model_name, load_in_8bit=True, device_map="auto")


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/9.80G [00:00<?, ?B/s]

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/9.93G [00:00<?, ?B/s]

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/9.93G [00:00<?, ?B/s]

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/2.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

### Sandbox

In [11]:
# img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg' 
# raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

# question = "give a caption describing the action of the dog"

# # Image captioning (without providing a text prompt):
# inputs = processor(raw_image, question, return_tensors="pt").to("cuda", torch.float16)

# out = model.generate(**inputs)
# print(processor.decode(out[0], skip_special_tokens=True))

playing with a woman on the beach


In [7]:
# img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg' 
# raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

# # Image captioning (without providing a text prompt):
# inputs = processor(raw_image, return_tensors="pt").to("cuda", torch.float16)

# out = model.generate(**inputs)
# print(processor.decode(out[0], skip_special_tokens=True))

woman playing with dog on beach


In [5]:
# Half Precision
# processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl", torch_dtype=torch.float16, device_map="auto")

# img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg' 
# raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

# question = "how many dogs are in the picture?"
# inputs = processor(raw_image, question, return_tensors="pt").to("cuda", torch.float16)

# Full Precision
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl", device_map="auto")
# question = "how many dogs are in the picture?"
# inputs = processor(raw_image, question, return_tensors="pt").to("cuda")


### Winoground

In [2]:
from dotenv import load_dotenv
import os
from tqdm import tqdm
import json

load_dotenv()  # This loads the environment variables from .env

api_key = os.getenv('HGF_KEY')

In [3]:
from datasets import load_dataset
winoground = load_dataset('facebook/winoground', token=api_key)['test']

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
SAMPLE_SIZE=5
NUM_BEAMS=5
DO_SAMPLE=True

In [11]:
example = winoground[0]
# winoground dataset images are already in RGB mode
raw_image_0 = example['image_0'] 
# Image captioning (without providing a text prompt):
inputs = processor(raw_image_0, return_tensors="pt").to("cuda", torch.float16)

out = model.generate(**inputs,do_sample=True,num_beams=5,num_return_sequences=SAMPLE_SIZE,
temperature=1)
print(processor.decode(out[0], skip_special_tokens=True))

a man in a cowboy hat kissing a little girl on the cheek



In [10]:
for o in out:
    print(processor.decode(o, skip_special_tokens=True))

a man in a cowboy hat kissing a little girl on the cheek

a man in cowboy hat kissing a little girl on the cheek

a man in cowboy hat kissing a little girl on the cheek

a man in a cowboy hat kissing a young girl on the cheek

a man in cowboy hat kissing a young girl on the cheek



In [12]:
def generate_on_data(winoground,f_stream):
    for example in tqdm(winoground):
        d = {}
        d['id'] = example['id']
    
        for i in range(2):     
            raw_image = example[f'image_{i}'] 
            inputs = processor(raw_image, return_tensors="pt").to("cuda", torch.float16)
            out = model.generate(**inputs,do_sample=DO_SAMPLE,
                                   num_beams=NUM_BEAMS,num_return_sequences=SAMPLE_SIZE)
            d[f'caption_{i}'] = [processor.decode(o, skip_special_tokens=True) for o in out]
    
        print(json.dumps(d),file=f_stream,flush=True)

In [7]:
with open(f'wino_{model_name}_caps.jsonl','w') as f:
    generate_on_data(winoground,f)

  0%|                                                                  | 0/400 [00:00<?, ?it/s]/mnt/efs/fs1/mmml/miniconda3/envs/cenv_x86/lib/python3.8/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|████████████████████████████████████████████████████████| 400/400 [35:07<00:00,  5.27s/it]


In [3]:
import smatch

In [4]:
help(smatch)

Help on module smatch:

NAME
    smatch

DESCRIPTION
    This script computes smatch score between two AMRs.
    For detailed description of smatch, see http://www.isi.edu/natural-language/amr/smatch-13.pdf

FUNCTIONS
    compute_f(match_num, test_num, gold_num)
        Compute the f-score based on the matching triple number,
                                     triple number of AMR set 1,
                                     triple number of AMR set 2
        Args:
            match_num: matching triple number
            test_num:  triple number of AMR 1 (test file)
            gold_num:  triple number of AMR 2 (gold file)
        Returns:
            precision: match_num/test_num
            recall: match_num/gold_num
            f_score: 2*precision*recall/(precision+recall)
    
    compute_match(mapping, weight_dict)
        Given a node mapping, compute match number based on weight_dict.
        Args:
        mappings: a list of node index in AMR 2. The ith element (value j) mea